# How to Serve MistralFlite on TGI

## Start TGI Server

In [155]:
!mkdir -p models

In [156]:
!docker run -d --gpus all --shm-size 1g -p 443:80 -v $(pwd)/models:/data ghcr.io/huggingface/text-generation-inference:1.1.0 \
      --model-id amazon/MistralLite \
      --max-input-length 16000 \
      --max-total-tokens 16384 \
      --max-batch-prefill-tokens 16384 \
      --trust-remote-code

2a0cfd4e6a78231f9811edef810ee3a0360cae774a04ec56bb6073b742687bed


> Warning: You may need to wait for 10+ minutes for the docker container to be ready for the first time.

## Perform Inference

In [7]:
!pip install text_generation==0.6.1

  Obtaining dependency information for text_generation from https://files.pythonhosted.org/packages/14/f7/cadf3a0fc619a72d7c667d16e96ef0a5b4c557e6e2b4788a0360dfba4fee/text_generation-0.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0,>=3.8 from https://files.pythonhosted.org/packages/41/8e/4c48881316bbced3d13089c4d0df4be321ce79a0c695d82dee9996aaf56b/aiohttp-3.8.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.4 MB/s eta 0:00:0000:01
  Obtaining dependency information for async-timeout<5.0,>=4.0.0a3 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.6 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for frozenlist>=1.1.1 from https://files.pythonhosted.org/packages/1e/28/74b8b6451c89c070d34e753d8

In [157]:
from text_generation import Client

SERVER_PORT = 443
SERVER_HOST = "localhost"
SERVER_URL = f"{SERVER_HOST}:{SERVER_PORT}"
tgi_client = Client(f"http://{SERVER_URL}", timeout=60)

def invoke_tgi(prompt, 
                      random_seed=1, 
                      max_new_tokens=400, 
                      print_stream=True,
                      assist_role=True):
    if (assist_role):
        prompt = f"<|prompter|>{prompt}</s><|assistant|>"
    output = ""
    for response in tgi_client.generate_stream(
        prompt,
        do_sample=False,
        max_new_tokens=max_new_tokens,
        return_full_text=False,
        #temperature=None,
        #truncate=None,
        #seed=random_seed,
        #typical_p=0.2,
    ):
        if hasattr(response, "token"):
            if not response.token.special:
                snippet = response.token.text
                output += snippet
                if (print_stream):
                    print(snippet, end='', flush=True)
    return output

In [159]:
prompt = "What are the main challenges to support a long context for LLM?"
result = invoke_tgi(prompt)

 There are several challenges to supporting a long context for LLMs, including:

1. Memory Limitations: LLMs are limited in the amount of data they can store in memory, which can limit the length of the context they can support.

2. Computational Complexity: Generating responses for long contexts can be computationally expensive, especially for larger models.

3. Training Data: Training data for long contexts can be difficult to obtain, as it requires a large amount of text data that is relevant and coherent.

4. Bias and Unanswerable Questions: Long contexts can increase the likelihood of bias and unanswerable questions, as the model may not have seen similar contexts during training.

5. Incomplete or Inconsistent Data: Long contexts can be more susceptible to errors and inconsistencies in the data, which can affect the accuracy of the model's responses.

6. Maintenance and Updating: Maintaining and updating a long context can be challenging, as it requires regular monitoring and upd

In [163]:
with open("../example_long_ctx.txt", "r") as fin:
    task_instruction = fin.read()
    task_instruction = task_instruction.format(
        my_question="please tell me how does pgvector help with Generative AI and give me some examples."
    )
prompt = f"<|prompter|>{task_instruction}</s><|assistant|>"
result = invoke_tgi(prompt)

GenerationError: Request failed during generation: Server error: Out of available cache blocks: asked 946, only 154 free blocks